In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#scvファイルの読み込み
livestock = pd.read_csv("../data/livestock.csv")
cultured = pd.read_csv("../data/cultured.csv")

In [3]:
data_pre = pd.concat([livestock,cultured],sort=False)
data = data_pre.reset_index(drop=True)

In [4]:
# 新しい説明変数を作成する

#一人当たり支援額
data["一人当たり支援額"] = round(data["支援総額"]/(data["支援者数"]+1))
#最高額支援ダミー:最高支援額と支援額上限が一致するか 0:不一致 1:一致
data["最高額支援ダミー"] = 0
data.loc[data['最高支援額']==data['支援額上限'],'最高額支援ダミー'] = 1
#最低額返礼品ダミー:支援額下限と返礼品ありの下限が一致するか　0:不一致　1:一致
data["最低額返礼品ダミー"] = 0
data.loc[data['支援額下限']==data['返礼品ありの下限'],'最低額返礼品ダミー'] = 1
#自然災害ダミー:災害を示すような言葉が含まれているか
data["自然災害ダミー"] = 0
for i in range(len(data)):
    if "震災" in data["プロジェクト名"][i]:
        data["自然災害ダミー"][i] = 1
    elif "豪雨" in data["プロジェクト名"][i]:
        data["自然災害ダミー"][i] = 1
    elif "台風" in data["プロジェクト名"][i]:
        data["自然災害ダミー"][i] = 1
#畜産ダミー:畜産か耕種か　畜産:1 耕種:0
data["畜産ダミー"] = 0
data["畜産ダミー"][:110] = 1

#target値を作成　0:失敗　1:成功
data["achievement"] = data["支援総額"] // data["目標金額"]
data["target"] = 0
data.loc[data['achievement']>=1,'target'] = 1

<ipython-input-4-c46add677b41>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["自然災害ダミー"][i] = 1
<ipython-input-4-c46add677b41>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["自然災害ダミー"][i] = 1
<ipython-input-4-c46add677b41>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["自然災害ダミー"][i] = 1
<ipython-input-4-c46add677b41>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



In [5]:
def hold_out(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
    model = sm.Logit(y_train, X_train)
    results = model.fit()
    pred = results.predict(X_test)
    result = [1 if i>0.5 else 0 for i in pred]
    y_test_re = y_test.reset_index(drop=True)
    count=0
    for i in range(len(y_test)):
        if y_test_re[i] == result[i]:
            count+=1
    return count/len(y_test)

# 全体のデータ

In [6]:
delate_col = ["プロジェクト名","URL","CFP","支援総額","産業","最高支援額","最低支援額","achievement"
              ,"一人当たり支援額"]
data_del_overall = data.drop(columns=delate_col)

In [7]:
data_del_overall

,目標金額,支援者数,画像数,動画数,文字数,活動報告回数,リターン種類数,支援額上限,支援額下限,返礼品ありの下限,最高額支援ダミー,最低額返礼品ダミー,自然災害ダミー,畜産ダミー,target
0,1000000,71,8,0,2566,3,9,50000,5000,5000,1,1,0,1,1
1,1000000,72,22,0,4010,5,12,100000,3000,3000,1,1,0,1,1
2,1000000,91,9,1,4244,27,12,100000,5000,5000,1,1,0,1,1
3,600000,79,10,0,3500,13,5,15000,5000,10000,0,0,0,1,1
4,1000000,116,18,0,3457,17,6,300000,3000,10000,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,150000,11,14,0,1156,3,5,15000,1000,1000,0,1,0,0,0
689,300000,59,21,0,4608,3,9,120000,500,500,1,1,0,0,1
690,500000,3,21,0,1320,0,5,48000,6000,6000,0,1,0,0,0
691,1080000,40,10,0,1870,5,7,108000,5400,5400,1,1,0,0,0


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
stratifiedfold = StratifiedKFold(n_splits=5)

In [9]:
y_overall = data_del_overall["target"]
x_overall = data_del_overall.drop("target",axis=1)

In [10]:
# logreg = LogisticRegression()
# scores = cross_val_score(logreg,x,y,cv=stratifiedfold)
# print(f'LogisticRegression:{scores.mean()}')

In [18]:
hold_out(x_overall,y_overall)

Optimization terminated successfully.
         Current function value: 0.336265
         Iterations 9


0.8273381294964028

In [11]:
import statsmodels.api as sm
X_overall = sm.add_constant(x_overall)
model = sm.Logit(y_overall, X_overall)
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.319460
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                  693
Model:                          Logit   Df Residuals:                      678
Method:                           MLE   Df Model:                           14
Date:                Wed, 23 Dec 2020   Pseudo R-squ.:                  0.5382
Time:                        18:43:41   Log-Likelihood:                -221.39
converged:                       True   LL-Null:                       -479.36
Covariance Type:            nonrobust   LLR p-value:                3.840e-101
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.9414      0.489     -6.020      0.000      -3.899      -1.984
目標金額       -1.971e-06    2.1e

# 耕種農業

In [12]:
delate_col = ["プロジェクト名","URL","CFP","支援総額","産業","最高支援額","最低支援額","achievement"
              ,"一人当たり支援額","畜産ダミー"]
data_delate_liv_dummy = data.drop(columns=delate_col)

In [13]:
livestock = data_delate_liv_dummy[:109]
cultured = data_delate_liv_dummy[109:]

In [14]:
y_cultured = cultured["target"]
x_cultured = cultured.drop("target",axis=1)

In [20]:
hold_out(x_cultured,y_cultured)

Optimization terminated successfully.
         Current function value: 0.303238
         Iterations 9


0.8888888888888888

In [15]:
X_cultured = sm.add_constant(x_cultured)
model = sm.Logit(y_cultured, X_cultured)
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.302751
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                  584
Model:                          Logit   Df Residuals:                      570
Method:                           MLE   Df Model:                           13
Date:                Wed, 23 Dec 2020   Pseudo R-squ.:                  0.5631
Time:                        18:43:41   Log-Likelihood:                -176.81
converged:                       True   LL-Null:                       -404.67
Covariance Type:            nonrobust   LLR p-value:                 3.603e-89
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.8385      0.536     -5.292      0.000      -3.890      -1.787
目標金額       -2.288e-06   2.78e

# 畜産

In [16]:
y_livestock = livestock["target"]
x_livestock = livestock.drop("target",axis=1)

In [21]:
hold_out(x_livestock,y_livestock)

         Current function value: 0.366023
         Iterations: 35


/Users/takato/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


0.7727272727272727

In [17]:
X_livestock = sm.add_constant(x_livestock)
model = sm.Logit(y_livestock, X_livestock)
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.264417
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                  109
Model:                          Logit   Df Residuals:                       95
Method:                           MLE   Df Model:                           13
Date:                Wed, 23 Dec 2020   Pseudo R-squ.:                  0.6033
Time:                        18:43:41   Log-Likelihood:                -28.821
converged:                       True   LL-Null:                       -72.660
Covariance Type:            nonrobust   LLR p-value:                 3.879e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.8033      3.500     -3.372      0.001     -18.664      -4.943
目標金額       -1.616e-06   4.19